In [1]:
import numpy
import pandas

train_ud = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\train-ud_paths_per_relation_no_relation_with_tokens.csv')
test_ud = pandas.read_csv(r'C:\Users\JYellin\re_1\tacred\results\general\test-ud_paths_per_relation_no_relation_with_tokens.csv')


In [29]:
relations = train_ud['relation'].unique().tolist()

for relation in relations:
    type1 = train_ud[train_ud['relation'] == relation]['type1'].unique().tolist()
    type2 = train_ud[train_ud['relation'] == relation]['type2'].unique().tolist()
    print('Relation {}: type1 = {}; types2 = {}'.format(relation, type1, type2))

Relation org:founded_by: type1 = ['ORGANIZATION']; types2 = ['PERSON']
Relation no_relation: type1 = ['PERSON', 'ORGANIZATION']; types2 = ['PERSON', 'ORGANIZATION', 'NUMBER', 'DATE', 'NATIONALITY', 'LOCATION', 'TITLE', 'MISC', 'COUNTRY', 'CRIMINAL_CHARGE', 'CITY', 'DURATION', 'RELIGION', 'IDEOLOGY', 'CAUSE_OF_DEATH', 'STATE_OR_PROVINCE', 'URL']
Relation per:employee_of: type1 = ['PERSON']; types2 = ['ORGANIZATION', 'LOCATION']
Relation org:alternate_names: type1 = ['ORGANIZATION']; types2 = ['ORGANIZATION', 'MISC']
Relation per:cities_of_residence: type1 = ['PERSON']; types2 = ['CITY', 'LOCATION']
Relation per:children: type1 = ['PERSON']; types2 = ['PERSON']
Relation per:title: type1 = ['PERSON']; types2 = ['TITLE']
Relation per:siblings: type1 = ['PERSON']; types2 = ['PERSON']
Relation per:religion: type1 = ['PERSON']; types2 = ['RELIGION']
Relation per:age: type1 = ['PERSON']; types2 = ['NUMBER', 'DURATION']
Relation org:website: type1 = ['ORGANIZATION']; types2 = ['URL']
Relation p

In [65]:
path_counts = train_ud[train_ud['relation']!='no_relation'].groupby(['path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)
relation_counts = train_ud[train_ud['relation']!='no_relation'].groupby(['relation'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)

In [130]:
paths = path_counts['path'].tolist()
relations = relation_counts['relation'].tolist()
matrix_count = numpy.zeros((path_counts.shape[0], relation_counts.shape[0]), dtype=numpy.int32)
matrix_existence = numpy.zeros((path_counts.shape[0], relation_counts.shape[0]), dtype=numpy.int32)

In [124]:
helper = train_ud[train_ud['relation'] != 'no_relation'].groupby(['path','relation'])

def get_path_and_relation_count(path, relation):
    key = (path, relation)
    
    if key in helper.groups:
        return helper.get_group(key).shape[0]
    else:
        return 0

In [131]:
for path_index, path in enumerate(paths):
    for relation_index, relation in enumerate(relations):
        count = get_path_and_relation_count(path, relation)
        matrix_count[path_index,relation_index] = count
        matrix_existence[path_index,relation_index] = int(count>0)
        
        
        

In [134]:
path_to_relation_count = pandas.DataFrame(data=matrix_count, index=paths, columns=relations)
path_to_relation_existence = pandas.DataFrame(data=matrix_existence, index=paths, columns=relations)

In [137]:
path_to_relation_existence['sum'] = path_to_relation_existence.sum(axis=1)

In [144]:
path_to_relation_existence = path_to_relation_existence[path_to_relation_existence['sum'] > 1]

In [145]:
path_to_relation_existence

,per:title,org:top_members/employees,per:employee_of,org:alternate_names,org:country_of_headquarters,per:countries_of_residence,per:age,org:city_of_headquarters,per:cities_of_residence,per:stateorprovinces_of_residence,...,per:charges,per:city_of_birth,per:date_of_birth,per:religion,per:stateorprovince_of_death,per:stateorprovince_of_birth,per:country_of_birth,org:dissolved,per:country_of_death,sum
<compound >compound,1,1,1,0,1,1,0,1,1,0,...,0,0,0,0,1,0,0,0,0,10
<compound <nsubj <ROOT,1,0,1,0,1,1,0,0,1,1,...,0,0,0,0,0,0,1,0,1,13
<compound >amod,1,0,1,1,1,1,0,0,1,1,...,0,0,0,1,0,0,0,0,0,13
<det >appos >punct,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
>ROOT >nsubj >compound,1,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
<amod <dobj >compound:prt,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2
<appos >punct,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
>ROOT >nmod:in,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,2
<amod <dobj >nmod:in >case,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,2


In [129]:
path_counts[path_counts['total']>1]

,path,total
0,<compound >compound,430
1,<compound <nsubj <ROOT,254
2,<compound >amod,237
3,<det >appos >punct,143
4,>ROOT >nsubj >compound,141
...,...,...
1096,<amod <dobj >nmod:in >case,2
1097,<acl:to <dobj <ccomp >nsubj >amod,2
1098,>ROOT >nmod:as >nmod:for >det,2
1099,<amod <appos <nsubj >punct,2
